<a href="https://colab.research.google.com/github/LilithUmberger/TurtleAerobiomeData/blob/Code-and-Output/File_Creation_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Be sure to set the runtime to run all in order to get everything necessary for the entire code!

# Before Processing Imports

In [25]:
#Importing needed modules

import csv
import os
from os import listdir
import shutil

In [26]:
#Removing all previous files and directories and starting fresh with the
#imported data and empty directories for created files

!rm *
for root, dirs, files in os.walk('.', topdown=False):
    for name in dirs:
        shutil.rmtree(os.path.join(root, name))
!git clone -b Samples https://github.com/LilithUmberger/TurtleAerobiomeData.git
!mkdir '/content/Twenty'
!mkdir '/content/PAMatrices'

rm: cannot remove '*': No such file or directory
Cloning into 'TurtleAerobiomeData'...
remote: Enumerating objects: 421, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 421 (delta 128), reused 190 (delta 90), pack-reused 146 (from 1)
Receiving objects: 100% (421/421), 1.56 MiB | 12.04 MiB/s, done.
Resolving deltas: 100% (180/180), done.


# Top Twenty Microbiota Each Sample

In [27]:
#Needed variables

collected = []
filtered = []
toplis = []
sublis = []
filis = []
ficomp = []

In [28]:
#This function opens the files that are given to it through the directory, takes
#out the names, percents, and reads, sorts them by reads, removes items that are
#not names of the microbiota of interest, and creates new files with the names,
#percents, and reads of the top twenty of the sample

def filecreate(filename,collected,filtered,ficomp):
  working = open(filename,'r')
  for line in working:
    cols = line.split()
    percent = cols[1]
    read = cols[2]
    name = ''
    for word in range(len(cols[4:])):
      name = name + ' ' + cols[word+4]
    collected.append((name,percent,read))
    name = ''
  working.close()
  titles = collected[0]
  collected.remove(titles)
  collected.sort(key=lambda tup: int(tup[2]),reverse=True)
  for item in collected:
    if 'unclassified' in item[0]:
      continue
    if 'cannot' in item[0]:
      continue
    if ('Viruses' in item[0]) and ('Bac' in filename):
      continue
    filtered.append(item)
  collected = filtered
  split = filename.split('/')
  filnam = split[4]
  fidir = f'/content/Twenty/{filnam}.csv'
  ficomp.append(fidir)
  writing = open(fidir,'w', encoding='UTF8', newline='')
  writer = csv.writer(writing)
  writer.writerow(titles)
  for tup in range(0,20):
    linewrite = list(collected[tup])
    writer.writerow(linewrite)
  writing.close()

In [29]:
#This goes through the imported directory and looks at all the files in that
#directory, using the function mentioned above, to create a top twenty
#microbiota list for each sample as a csv file

directory = '/content/TurtleAerobiomeData'
for topfolder in os.listdir(directory):
  if ('.git' not in topfolder):
    topfoldername = directory + '/' + topfolder
    if topfoldername not in toplis:
      toplis.append(topfoldername)
      for subfolder in os.listdir(topfoldername):
        subfoldername = topfoldername + '/' + subfolder
        if subfoldername not in sublis:
          sublis.append(subfoldername)
          for file in os.listdir(subfoldername):
            filename = subfoldername + '/' + file
            if filename not in filis:
              filis.append(filename)
              filecreate(filename,collected,filtered,ficomp)
              collected = []
              filtered = []

# Presence Absence Matrices Each Sample

In [30]:
#This is taking all the files created in the previous step and is adding a short
#description, the name of the file, and the name of the microbiota of interest
#from the file

totalis = []
namelis = []

for dir in ficomp:
  hold = dir.split('/')
  fname = hold[3]
  dohold = fname.split('.')
  name = dohold[0]
  namelis.append(name)
  extract = open(dir,'r')
  next(extract)
  for line in extract:
    cols = line.split(',')
    if ('Bac' in name) and ('Turtle' in name):
      description = 'BT'
      tup = (description,name,cols[0])
      totalis.append(tup)
    elif ('Vir' in name) and ('Turtle' in name):
      description = 'VT'
      tup = (description,name,cols[0])
      totalis.append(tup)
    elif ('Bac' in name) and ('Air' in name):
      description = 'BA'
      tup = (description,name,cols[0])
      totalis.append(tup)
    elif ('Vir' in name) and ('Air' in name):
      description = 'VA'
      tup = (description,name,cols[0])
      totalis.append(tup)
  extract.close()

In [31]:
#This is making four lists using the descriptions added above

bt = []
vt = []
ba = []
va = []

for item in totalis:
  if (item[0] == 'BT') and (item[2] not in bt):
    bt.append(item[2])
  elif (item[0] == 'VT') and (item[2] not in vt):
    vt.append(item[2])
  elif (item[0] == 'BA') and (item[2] not in ba):
    ba.append(item[2])
  elif (item[0] == 'VA') and (item[2] not in va):
    va.append(item[2])

In [32]:
#This is going through the list that was created in the cell two up, looking at
#the list created above, and if the item in the first list is present in the
#second list, adding a 1 to the thing in the first list (if not a zero is added)

templis = []
edtot = []
for name in namelis:
  for item in totalis:
    if item[1] == name:
      templis.append(item[2])
  if ('Bac' in name) and ('Turtle' in name):
    for thing in bt:
      if thing in templis:
        tup = (name,thing,1)
        edtot.append(tup)
      else:
        tup = (name,thing,0)
        edtot.append(tup)
  elif ('Vir' in name) and ('Turtle' in name):
    for thing in vt:
      if thing in templis:
        tup = (name,thing,1)
        edtot.append(tup)
      else:
        tup = (name,thing,0)
        edtot.append(tup)
  elif ('Bac' in name) and ('Air' in name):
    for thing in ba:
      if thing in templis:
        tup = (name,thing,1)
        edtot.append(tup)
      else:
        tup = (name,thing,0)
        edtot.append(tup)
  elif ('Vir' in name) and ('Air' in name):
    for thing in va:
      if thing in templis:
        tup = (name,thing,1)
        edtot.append(tup)
      else:
        tup = (name,thing,0)
        edtot.append(tup)
  templis = []

In [33]:
#This block is taking each sample's data that was found in edtot and placing it
#into a file for processing with R

finam = []
reedtot = sorted(edtot, key=lambda x: x[1])
for name in namelis:
  fi = f'/content/PAMatrices/{name}PA.csv'
  writing = open(fi,'w', encoding='UTF8', newline='')
  writer = csv.writer(writing)
  title = ['Name','P/A']
  writer.writerow(title)
  for item in reedtot:
    if item[0] == name:
      tup = (item[1],item[2])
      row = list(tup)
      writer.writerow(row)
  writing.close()
  finam.append(fi)

# Making Folders Downloadable

In [34]:
#This block compresses the created directories into zip files for downloading

twenty = '/content/Twenty'
pam = '/content/PAMatrices'
zip_path1 = '/content/Twenty.zip'
zip_path2 = '/content/PAMatrices.zip'

shutil.make_archive(zip_path1.replace('.zip', ''), 'zip', twenty)
shutil.make_archive(zip_path2.replace('.zip', ''), 'zip', pam)

'/content/PAMatrices.zip'